# <center>Projet d'optimisation pour la classification </center>

Ce notebook détaille les étapes de la construction d'un réseau de neurones pour la classification.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
# import matplotlib  # maybe useful Ubuntu rodo ISIMA
# matplotlib.use('Qt5Agg') # maybe useful Ubuntu rodo ISIMA
import csv

from typing import Callable, List

from optimcourse.gradient_descent import gradient_descent, gradient_finite_diff
from optimcourse.optim_utilities import print_rec
from optimcourse.forward_propagation import (
    forward_propagation,
    create_weights,
    vector_to_weights,
    weights_to_vector)
from optimcourse.activation_functions import (
    relu,
    sigmoid,
    linear,
    leaky_relu
)
from optimcourse.test_functions import (
    linear_function,
    ackley,
    sphere,
    quadratic,
    rosen,
    L1norm,
    sphereL1
)
from optimcourse.restarted_gradient_descent import restarted_gradient_descent


## Importation des données



In [ ]:
# read data file
with open('./donnees_mensuration.csv', mode='r', encoding='utf-8') as file:
    csv_reader = csv.reader(file,delimiter=';')
    header = next(csv_reader)
    meas_data = []
    for row in csv_reader:
        # Convert numeric strings to float where applicable
        processed_row = []
        for item in row:
            try:
                # Attempt to convert to float
                processed_row.append(float(item))
            except ValueError:
                # If conversion fails, keep it as a string
                processed_row.append(item)
        meas_data.append(processed_row)


## Contruction des entrées et sorties
Ici, les entrées sont les mensurations, et les sorties le genre de la personne

In [ ]:
# create inputs
n_obs = len(meas_data)
n_features = 6 # poids taille ventre hanche epaule chaussure
inputs = np.ndarray((n_obs,n_features))
inputs[:,0:6]=[row[2:8] for row in meas_data[:]]
# outputs: male=0, female=1
gender_col = [row[1] for row in meas_data[:]]
gender = np.array([0 if g=='Masculin' else 1 for g in gender_col])
# put them in a dictionary
gender_data={"data":inputs,"target":gender}

## La famille des fonctions de perte
Ci-dessous une collection de fonctions qui servent à générer les fonctions de perte (loss functions). On se sert des variables globales pour passer des paramètres aux fonctions sans qu'ils figurent dans les arguments, ce qui permet de créer une fonction `f(x)` en passant en plus de `x` la structure du réseau ... La variable `__is_logistic__` est aussi globale et sert à ajouter un traitement logistique (par la fonction sigmoïde) en sortie de réseau de neurones. 

In [ ]:

# mean squared error
def cost_function_mse(y_predicted: np.ndarray,y_observed: np.ndarray):
    error = 0.5 * np.mean((y_predicted - y_observed)**2)
    return error


# entropy
# TODO : make it more robust by testing when y_predicted is equal or less than 0, or equal or larger than 1 and
#        by not performing the multiplication for the null terms
def cost_function_entropy(y_predicted: np.ndarray,y_observed: np.ndarray):

    n = len(y_observed)
    
    term_A = np.multiply(np.log(y_predicted),y_observed)
    term_B = np.multiply(1-y_observed,np.log(1-y_predicted))
    
    error = - (1/n)*(np.sum(term_A)+np.sum(term_B))

    return(error)


# TODO: I think this function would only work for 1 output because of the reshape(-1) at the end that is not applied to the data.
# --> make it multi-dimensional
def error_with_parameters(vector_weights: np.ndarray,
                          network_structure: List[int],
                          activation_function: Callable,
                          data: dict,
                          cost_function: Callable,
                          regularization: float = 0) -> float:
    
    weights = vector_to_weights(vector_weights,used_network_structure)
    predicted_output = forward_propagation(data["data"],weights,activation_function,logistic=__is_logistic__)
    predicted_output = predicted_output.reshape(-1,)
    
    error = cost_function(predicted_output,data["target"]) + regularization * np.sum(np.abs(vector_weights))
    
    return error


def neural_network_cost(vector_weights):
    cost = error_with_parameters(vector_weights,
                                 network_structure=used_network_structure,
                                 activation_function=used_activation,
                                 data=used_data,
                                 cost_function=used_cost_function)

    return cost


## Construction du réseau de neurones


In [ ]:
# ### Create the network
# and calculate the cost function of the first, randomly initialized, network.

used_network_structure = [6,5,1]
used_activation = leaky_relu 
# other examples of activation functions descriptions:
#  [[sigmoid,sigmoid,sigmoid,leaky_relu,leaky_relu],[leaky_relu]]
#  [sigmoid,leaky_relu] 
#  sigmoid or leaky_relu or sigmoid
used_data = gender_data # simulated_data
used_cost_function = cost_function_entropy # cost_function_mse
__is_logistic__ = True
weights = create_weights(used_network_structure)
weights_as_vector,_ = weights_to_vector(weights)
dim = len(weights_as_vector) 
###  a forward propagation
# predicted_output = forward_propagation(inputs=simulated_data["data"],weights=weights,activation_functions=relu,logistic=True)
# print(predicted_output)
print("Number of weights to learn : ",dim)
print("Initial cost of the NN : ",neural_network_cost(weights_as_vector))


## Apprentissage du réseau

par minimisation de la fonction perte ou `cost_function_...`

In [ ]:
LB = [-8] * dim
UB = [8] * dim
printlevel = 1
# res = gradient_descent(func = neural_network_cost,
#                  start_x = weights_as_vector,
#                  LB = LB, UB = UB,budget = 100,printlevel=printlevel,
#                  min_step_size = 1e-13, min_grad_size = 1e-13, do_linesearch=True,step_factor=0.01, direction_type="momentum"
#             )
#

res = restarted_gradient_descent(func=neural_network_cost, start_x=weights_as_vector,LB=LB,UB=UB,budget=1000,nb_restarts=1,
                                 printlevel=printlevel)
print_rec(res=res, fun=neural_network_cost, dim=len(res["x_best"]),
           LB=LB, UB=UB , printlevel=printlevel, logscale = True)

weights_best = vector_to_weights(res["x_best"],used_network_structure)

# a small study about the gradients ...
# initial gradient
init_grad = gradient_finite_diff(func=neural_network_cost , x=weights_as_vector , f_x=neural_network_cost(weights_as_vector))
final_grad = gradient_finite_diff(func=neural_network_cost , x=res["x_best"] , f_x=neural_network_cost(res["x_best"]))
#


print("Best NN weights:",weights_best)


# **THE END**